In [24]:
import os

DATADIR = ""
DATAFILE = "beatles-diskography.csv"


def parse_file(datafile):
    data = []
    with open(datafile, "r") as f:
        header = f.readline().split(",")
        counter = 0
        for line in f:
            if counter == 10:
                break

            fields = line.split(",")
            entry = {}

            for key, value in enumerate(fields):
                entry[header[key].strip()] = value.strip()
            if counter==5:
                print "line"
                print line
                print "fields"
                print fields
                print "entry"
                print entry
            data.append(entry)
            counter += 1

    return data

def test():

    datafile = os.path.join(DATADIR, DATAFILE)
    d = parse_file(datafile)
   

test()



line
Twist and Shout,3 February 1964,Capitol(CAN),-,-,,

fields
['Twist and Shout', '3 February 1964', 'Capitol(CAN)', '-', '-', '', '\r\n']
entry
{'Title': 'Twist and Shout', 'UK Chart Position': '-', 'Label': 'Capitol(CAN)', 'Released': '3 February 1964', 'US Chart Position': '-', 'RIAA Certification': '', 'BPI Certification': ''}


In [25]:
import os
import csv

# os.getcwd()
# chdir(...)
# os.chdir('desired path')
# f = open("beatles-diskography.csv", "rb")
# header = f.readline().split(",")
# l = f.readline()

DATADIR = os.getcwd()
DATAFILE = "beatles-diskography.csv"


def parse_file(datafile):
    data = []
    with open(datafile, "rb") as f:
        r = csv.DictReader(f)
        for line in r:
            data.append(line)
    return data

def test():
    datafile = os.path.join(DATADIR, DATAFILE)
    d = parse_file(datafile)
    
test()

[{'Title': 'Please Please Me', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '22 March 1963', 'US Chart Position': '-', 'RIAA Certification': 'Platinum', 'BPI Certification': 'Gold'}, {'Title': 'With the Beatles', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '22 November 1963', 'US Chart Position': '-', 'RIAA Certification': 'Gold', 'BPI Certification': 'Platinum'}, {'Title': 'Beatlemania! With the Beatles', 'UK Chart Position': '-', 'Label': 'Capitol(CAN)', 'Released': '25 November 1963', 'US Chart Position': '-', 'RIAA Certification': '', 'BPI Certification': ''}, {'Title': 'Introducing... The Beatles', 'UK Chart Position': '-', 'Label': 'Vee-Jay(US)', 'Released': '10 January 1964', 'US Chart Position': '2', 'RIAA Certification': '', 'BPI Certification': ''}, {'Title': 'Meet the Beatles!', 'UK Chart Position': '-', 'Label': 'Capitol(US)', 'Released': '20 January 1964', 'US Chart Position': '1', 'RIAA Certification': '5xPlatinum', 'BPI Certificat

In [29]:
import xlrd

datafile = "2013_ERCOT_Hourly_Load_Data.xls"


def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)

    data = [[sheet.cell_value(r, col) 
                for col in range(sheet.ncols)] 
                    for r in range(sheet.nrows)]

    print "\nList Comprehension"
    print "data[3][2]:",
    print data[3][2]

    print "\nCells in a nested loop:"    
    for row in range(sheet.nrows):
        for col in range(sheet.ncols):
            if row == 2:
                print sheet.cell_value(row, col),


    ### other useful methods:
    print "\nROWS, COLUMNS, and CELLS:"
    print "Number of rows in the sheet:", 
    print sheet.nrows
    print "Type of data in cell (row 3, col 2):", 
    print sheet.cell_type(3, 2)
    print "Value in cell (row 3, col 2):", 
    print sheet.cell_value(3, 2)
    print "Get a slice of values in column 3, from rows 1-3:"
    print sheet.col_values(3, start_rowx=1, end_rowx=4)

    print "\nDATES:"
    print "Type of data in cell (row 1, col 0):", 
    print sheet.cell_type(1, 0)
    exceltime = sheet.cell_value(1, 0)
    print "Time in Excel format:",
    print exceltime
    print "Convert time to a Python datetime tuple, from the Excel float:",
    print xlrd.xldate_as_tuple(exceltime, 0)

    return data

data = parse_file(datafile)


List Comprehension
data[3][2]: 1036.088697

Cells in a nested loop:
41275.0833333 7388.082714 1035.021938 1403.472287 776.307387 10152.358518 2159.733208 4233.587967 872.40475 28020.968769 
ROWS, COLUMNS, and CELLS:
Number of rows in the sheet: 7296
Type of data in cell (row 3, col 2): 2
Value in cell (row 3, col 2): 1036.088697
Get a slice of values in column 3, from rows 1-3:
[1411.7505669999982, 1403.4722870000019, 1395.053150000001]

DATES:
Type of data in cell (row 1, col 0): 3
Time in Excel format: 41275.0416667
Convert time to a Python datetime tuple, from the Excel float: (2013, 1, 1, 1, 0, 0)


In [31]:
import json
import requests

BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"


query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):

    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print "requesting", r.url

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):

    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):

    if type(data) == dict:
        print json.dumps(data, indent=indent, sort_keys=True)
    else:
        print data


def main():

    results = query_by_name(ARTIST_URL, query_type["simple"], "Nirvana")
    pretty_print(results)

    print "\nARTIST:"
    pretty_print(results["artists"][3])

    artist_id = results["artists"][3]["id"]
    artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
    releases = artist_data["releases"]

    print "\nONE RELEASE:"
    pretty_print(releases[0], indent=2)

    release_titles = [r["title"] for r in releases]
    print "\nALL TITLES:"
    for t in release_titles:
        print t

if __name__ == '__main__':
    main()
    
    

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ANirvana&fmt=json
http://musicbrainz.org/ws/2/artist/  {'query': 'artist:Nirvana', 'fmt': 'json'}
{
    "artists": [
        {
            "area": {
                "id": "6a264f94-6ff1-30b1-9a81-41f7bfabd616", 
                "name": "Finland", 
                "sort-name": "Finland"
            }, 
            "country": "FI", 
            "disambiguation": "Early 1980's Finnish punk band", 
            "id": "85af0709-95db-4fbc-801a-120e9f4766d0", 
            "life-span": {
                "ended": null
            }, 
            "name": "Nirvana", 
            "score": "100", 
            "sort-name": "Nirvana", 
            "tags": [
                {
                    "count": 1, 
                    "name": "punk"
                }, 
                {
                    "count": 1, 
                    "name": "finland"
                }
            ], 
            "type": "Group"
        }, 
        {
          

In [62]:
import xml.etree.cElementTree as ET
import re
from collections import defaultdict

myOSMfile = "lausanne_switzerland.osm"

regex = re.compile(r'\b\S+\.?', re.IGNORECASE)
expected = ["Avenue", "Allée", "Route", "Rue", "Chemin","Place","Galeries","Jetée","Sentier",
            "Sur","Villas","Ruelle"] 

unknown_streets = defaultdict(set)
osm_file = open(myOSMfile, "r")

for event, elem in ET.iterparse(osm_file, events=("start",)):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
                if (tag.attrib['k'] == "addr:street"):
                    m = regex.search(tag.attrib['v'])
                    if m.group() not in expected:
                        unknown_streets[m.group()].add(tag.attrib['v'])
                
                    
                
for line in unknown_streets:
    print line


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:19: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


Boulevard
Vallée-de-la-Jeunesse
Passage
Jetée
ISDC
Grand-Rue
avenue
Vieux-Port
Guiguer-de-Prangins
Voie
en
Promenade
Av.
chemin
Quai
Coin
Escaliers
Jardins-de-Prélaz
Tunnel
Passerelle
Ch.
rue
Campagne
Parc
Figuiers
route
Grand'Rue
Quartier
place
Allée
